In [ ]:
# @title Install dependencies
!pip install jupyterlab-widgets geopandas geoplot

In [ ]:
# @title Select the whl file for the wawbus package
from ipywidgets import FileUpload
from IPython.display import display
upload = FileUpload(accept='.whl', multiple=False)
display(upload)

In [ ]:
# @title upload whl file
for uploaded_filename in upload.value:
  content = upload.value[uploaded_filename]['content']
  with open(uploaded_filename, "w+b") as f:
    f.write(content)

In [ ]:
# @title Install the wawbus package
!pip install {list(upload.value.keys())[0]}

In [ ]:
from wawbus import WawBus
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import matplotlib as mpl
import geoplot as gplt

from ipywidgets import Dropdown, Text, VBox, IntSlider

In [ ]:
# @title Main api configuration
dataset = Dropdown(options=['20240207-66k', 'None'], description='Prefetched dataset')
api_key = Text(description='API key (optional)', placeholder='Enter your API key')
collection = IntSlider(description='How many data points to collect. If zero, only the prefetched dataset will be used', min=0, step=1)
display(VBox([dataset, api_key, collection]))

In [ ]:
api = WawBus(apikey=api_key.value, dataset=dataset.value if dataset.value != 'None' else None)

In [ ]:
if collection.value > 0:
    api.collect_positions(collection.value)

In [ ]:
api.dataset.head()

In [ ]:
df = api.calculate_speed()
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Lon, df.Lat, crs='EPSG:4326'))
gdf.head()

In [ ]:
speeding = gdf[(gdf.Speed > 50) & (gdf.Speed < 120)]

In [ ]:
# @title first 3000 entries of >50km/h interactive map
speeding.head(3000)[['Speed', 'geometry', 'VehicleNumber', 'Lines']].explore(
    column='Speed',
    style_kwds={"style_function":lambda x: {"radius":x["properties"]["Speed"]/5 - 4}}
)

In [ ]:
wawmap = gpd.read_file('https://raw.githubusercontent.com/C10udburst/wawbus-data/master/warszawa-dzielnice.geojson')

In [ ]:
speed_map = gpd.sjoin(left_df=speeding, right_df=wawmap[['name', 'geometry']], how="left", predicate="intersects")
speed_map = speed_map.drop(columns=['index_right'])
speed_map.head()

In [ ]:
# @title >50km/h heatmap
mpl.rcParams['figure.dpi'] = 200

# we filter to only use entries within Warsaw
speeding_warsaw = speed_map.dropna().geometry

webmap = gplt.webmap(speeding_warsaw, projection=gcrs.WebMercator())
ax = gplt.polyplot(wawmap, projection=gcrs.WebMercator(), zorder=1, ax=webmap)
heatmap = gplt.kdeplot(speeding_warsaw, cmap='Reds', shade=True, ax=ax, projection=gcrs.WebMercator(), alpha=0.4)
plt.show()

In [ ]:
# @title >50 km/h per district
speed_map.groupby('name')['Speed'].count()